In [ ]:
# %pip install "peft==0.13.1" "diffusers==0.26.0" "huggingface_hub==0.24.0" "diffusers==0.26.0" transformers accelerate
%pip install peft==0.14.0 diffusers==0.32.2 transformers==4.48.0

In [3]:
# Full tune 生成图片示例代码

import torch
from diffusers import StableDiffusionXLPipeline

# 路径指向你那个 6G 的文件
model_path = "./outputs/new model.safetensors"
output_name = "liying_zhao_5090_test_full_tune.png"

print("🚀 5090 正在加载全量自定义模型 (SDXL)...")

# 注意：这里改用 from_single_file 方法加载 6G 的 safetensors
pipe = StableDiffusionXLPipeline.from_single_file(
    model_path,
    torch_dtype=torch.bfloat16,
    use_safetensors=True
).to("cuda")

# prompt = "A high-quality portrait of EWOLiyingZhao, 8k"
prompt = "EWOLiyingZhao with long hair, wearing a blue shirt. She is posing for the camera with her hands on her cheeks, giving off a playful and confident vibe. The woman appears to be smiling, possibly for the camera. The background of the image is blurry, which helps to focus on the subject."

print("⚡ 5090 正在推理...")
with torch.inference_mode():
    image = pipe(
        prompt=prompt,
        num_inference_steps=30,
        guidance_scale=7.5,
        width=1024,
        height=1024
    ).images[0]

image.save(output_name)
print(f"✅ 成功！全量模型生成的图片已保存: {output_name}")

🚀 5090 正在加载全量自定义模型 (SDXL)...


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 13.63it/s]


⚡ 5090 正在推理...


100%|██████████| 30/30 [00:02<00:00, 11.54it/s]


✅ 成功！全量模型生成的图片已保存: liying_zhao_5090_test_full_tune.png


# Lora 生成图片示例代码 with SDXL

In [ ]:
import torch
import os
from diffusers import StableDiffusionXLPipeline

# --- 1. 自动路径纠错逻辑 ---
def get_lora_path():
    candidates = [
        "./outputs/SD15_LiyingZhao_v1.safetensors",
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

LORA_PATH = get_lora_path()
BASE_MODEL = "./models/sd_xl_base_1.0.safetensors"
TRIGGER_WORD = "EWOLiyingZhao"
# PROMPT = f"A high-quality portrait of {TRIGGER_WORD}, traditional Chinese dress, cinematic lighting, 8k"
PROMPT = f"{TRIGGER_WORD} with long hair, wearing a blue shirt. She is posing for the camera with her hands on her cheeks, giving off a playful and confident vibe. The woman appears to be smiling, possibly for the camera. The background of the image is blurry, which helps to focus on the subject."
OUTPUT_NAME = "liying_zhao_5090_test_lora.png"

def run_inference():
    if not LORA_PATH:
        print("❌ 错误：找不到 LoRA 文件！请确认 last.safetensors 的位置。")
        return

    print(f"🚀 找到权重: {LORA_PATH}")
    print("⏳ 5090 加载模型中 (BFloat16)...")
    
    # 针对 5090 优化
    pipe = StableDiffusionXLPipeline.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.bfloat16,
        use_safetensors=True
    ).to("cuda")

    print(f"🎨 注入 LoRA...")
    # 如果这行报错，请先执行 pip install peft
    pipe.load_lora_weights(LORA_PATH)

    print("⚡ 5090 Blackwell 架构全速出图中...")
    with torch.inference_mode():
        image = pipe(
            prompt=PROMPT,
            num_inference_steps=30,
            guidance_scale=7.5,
            width=1024,
            height=1024
        ).images[0]

    image.save(OUTPUT_NAME)
    print(f"✅ 成功！图片已保存至: {OUTPUT_NAME}")

if __name__ == "__main__":
    run_inference()

/home/yusaku/miniconda3/envs/kohya_ss/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-19 20:05:32.228870: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-19 20:05:32.632041: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-19 20:05:32.632125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-19 20

❌ 错误：找不到 LoRA 文件！请确认 last.safetensors 的位置。


In [ ]:
# SD 1.5 LoRA 生成图片示例代码

import torch
import os
from diffusers import StableDiffusionPipeline  # 修正：从 XL 换回 1.5 对应的 Pipeline

# --- 1. 配置路径 ---
LORA_PATH = "./outputs/SD15_LiyingZhao_v1.safetensors"
# 修正：使用 SD 1.5 官方模型作为底模，与 LoRA 匹配
BASE_MODEL = "runwayml/stable-diffusion-v1-5" 
TRIGGER_WORD = "EWOLiyingZhao"
PROMPT = f"A photo of {TRIGGER_WORD}, wearing a blue shirt, smiling, 8k resolution, highly detailed."
OUTPUT_NAME = "liying_zhao_sd15_test.png"

def run_inference():
    if not os.path.exists(LORA_PATH):
        print("❌ 错误：找不到 LoRA 文件！")
        return

    print(f"🚀 5090 加载 SD 1.5 底模...")
    
    # 加载 SD 1.5 管道
    pipe = StableDiffusionPipeline.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16, # SD 1.5 通常用 float16 就很稳
        use_safetensors=True
    ).to("cuda")

    print(f"🎨 注入 SD 1.5 LoRA...")
    # 注入刚才训练好的 512 分辨率 LoRA
    pipe.load_lora_weights(LORA_PATH)

    print("⚡ 5090 极速生成中 (SD 1.5 在 5090 上通常只需 1 秒)...")
    with torch.inference_mode():
        image = pipe(
            prompt=PROMPT,
            negative_prompt="(worst quality, low quality:1.4), deformed, blurry",
            num_inference_steps=25,
            guidance_scale=7.5,
            width=512,  # 修正：SD 1.5 建议用 512
            height=512
        ).images[0]

    image.save(OUTPUT_NAME)
    print(f"✅ 成功！图片已保存至: {os.path.abspath(OUTPUT_NAME)}")

if __name__ == "__main__":
    run_inference()

🚀 5090 加载 SD 1.5 底模...


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  7.51it/s]


🎨 注入 SD 1.5 LoRA...
⚡ 5090 极速生成中 (SD 1.5 在 5090 上通常只需 1 秒)...


100%|██████████| 25/25 [00:00<00:00, 31.72it/s]


✅ 成功！图片已保存至: /home/yusaku/Workspace/人脸FineTuneWorkspace/kohya_ss/liying_zhao_sd15_test.png


# LoRA生成图片with DreamBooth

In [2]:
import torch
from diffusers import StableDiffusionXLPipeline

# 1. 路径设置
model_path = "./outputs/SDXL_DB_LYZ_Resume.ckpt" # 确保后缀正确

# 2. 初始化 Pipeline
# 我们直接加载单文件权重 (single_file_load)
pipe = StableDiffusionXLPipeline.from_single_file(
    model_path, 
    torch_dtype=torch.float16, 
    use_safetensors=True
)

# 3. 针对 5090 开启加速
pipe.to("cuda")
# 5090 支持高效的算子加速
# pipe.enable_xformers_memory_efficient_attention() # 如果你安装了 xformers
pipe.enable_attention_slicing() 

# 4. 设置 Prompt
# 记得加入你在训练时使用的触发词 "EWOLiyingZhao"
prompt = "A high-quality portrait of EWOLiyingZhao, a beautiful woman smiling, highly detailed, 8k wallpaper, cinematic lighting"
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality"

# 5. 执行生成
print("🚀 正在生成图片...")
# 一次生成10张图片
N = 10
for i in range(N):
    with torch.inference_mode():
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=30, # SDXL 建议 30-40 步
            guidance_scale=7.5,
            width=1024,
            height=1024
        ).images[0]

    # 6. 保存结果
    file_name = f"result_sdxl_{i}.png"
    image.save(file_name)
    print(f"✅ 图片已保存为 {file_name}")

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 17.58it/s]


🚀 正在生成图片...


100%|██████████| 30/30 [00:04<00:00,  6.92it/s]


✅ 图片已保存为 result_sdxl_0.png


100%|██████████| 30/30 [00:03<00:00,  7.77it/s]


...

✅ 图片已保存为 result_sdxl_7.png


100%|██████████| 30/30 [00:03<00:00,  7.78it/s]


✅ 图片已保存为 result_sdxl_8.png


100%|██████████| 30/30 [00:03<00:00,  7.84it/s]


✅ 图片已保存为 result_sdxl_9.png


In [ ]:
# Use as LoRA

import torch
from diffusers import StableDiffusionXLPipeline

# 配置文件路径
lora_path = "./outputs/SDXL_DB_LYZ_Resume_Extracted_LoRA.safetensors"
models = {
    "PonyV6": "./models/AstraliteHeart-pony-diffusion-v6.safetensors",
}

# 针对 5090 的通用 Prompt 配置
# 注意：Pony 模型通常需要特定的起手词才能达到最佳效果
prompt_map = {
    "PonyV6": "score_9, score_8_up, score_7_up, portrait of EWOLiyingZhao, beautiful woman, high quality, cinematic lighting",
}

negative_prompt = "lowres, bad anatomy, bad hands, text, error, blurry, deformed, watermark"

for name, model_path in models.items():
    print(f"🚀 正在加载模型: {name}...")
    
    # 1. 加载底模
    pipe = StableDiffusionXLPipeline.from_single_file(
        model_path, 
        torch_dtype=torch.float16,
        use_safetensors=True
    ).to("cuda")
    
    # 2. 挂载提取出来的 LoRA
    # adapter_name 可以随意命名，用于区分
    pipe.load_lora_weights(lora_path, adapter_name="lyz")
    
    # 3. 针对 5090 优化推理
    pipe.enable_attention_slicing()
    
    print(f"🎨 正在使用 {name} 生成图片...")
    with torch.inference_mode():
        image = pipe(
            prompt=prompt_map[name],
            negative_prompt=negative_prompt,
            num_inference_steps=35,
            guidance_scale=5.0, # Pony 建议降低 CFG
            width=1024,
            height=1024
        ).images[0]
    
    # 4. 保存结果
    output_filename = f"result_{name}_with_lora.png"
    image.save(output_filename)
    print(f"✅ 已保存: {output_filename}")
    
    # 5. 清理显存以便加载下一个模型
    del pipe
    torch.cuda.empty_cache()

print("\n✨ 所有测试完成！请检查当前目录下的 png 文件。")

# Lora 生成图片示例代码 with Flux

In [ ]:
# import torch
# import os
# from diffusers import FluxPipeline

# # --- 1. 配置路径 ---
# # 确保这是你训练脚本输出的 .safetensors 文件路径
# LORA_PATH = "./outputs/EWOLiyingZhao_Flux_v2_1024.safetensors"
# BASE_MODEL = "./models/FLUX.1-dev"
# TRIGGER_WORD = "EWOLiyingZhao"

# # 建议 Prompt 包含训练时的触发词
# PROMPT = f"{TRIGGER_WORD} with long hair, wearing a blue shirt, smiling, 8k resolution, cinematic lighting, masterpiece."
# OUTPUT_NAME = "liying_zhao_flux_final_test.png"

# def run_inference():
#     if not os.path.exists(LORA_PATH):
#         print(f"❌ 错误：在 ./outputs/ 没找到 LoRA 文件！")
#         return

#     print(f"🚀 5090 载入 LoRA: {LORA_PATH}")
#     print(f"⏳ 加载 Flux 基础模型中...")

#     # 加载 Flux 管道
#     pipe = FluxPipeline.from_pretrained(
#         BASE_MODEL,
#         torch_dtype=torch.bfloat16,
#         use_safetensors=True
#     )

#     # 🛠️ 针对 5090 和 64G 内存的优化策略
#     # 这一行会自动平衡 T5 编码器和 Transformer 的显存占用，彻底告别卡顿
#     pipe.enable_model_cpu_offload()

#     print(f"🎨 注入 LoRA 权重...")
#     pipe.load_lora_weights(LORA_PATH)

#     print("⚡ 5090 全速出图中...")
    
#     with torch.inference_mode():
#         # Flux 默认参数：1024x1024, 20-30 steps, guidance 3.5
#         image = pipe(
#             prompt=PROMPT,
#             height=1024,
#             width=1024,
#             guidance_scale=3.5, 
#             num_inference_steps=28,
#             max_sequence_length=512 # T5 处理长度
#         ).images[0]

#     image.save(OUTPUT_NAME)
#     print(f"✅ 成功！图片已保存至: {os.path.abspath(OUTPUT_NAME)}")

# if __name__ == "__main__":
#     run_inference()

🚀 5090 载入 LoRA: ./outputs/EWOLiyingZhao_Flux_v2_1024.safetensors
⏳ 加载 Flux 基础模型中...


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.23it/s]


🎨 注入 LoRA 权重...
⚡ 5090 全速出图中...


  0%|          | 0/28 [04:26<?, ?it/s]


KeyboardInterrupt: 

In [2]:
# 5090 Flux LoRA 推理示例代码（针对 32GB 显卡优化）

import torch
import os
import gc
from diffusers import FluxPipeline

# --- 1. 彻底冷启动清理 ---
def flush_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect() # 显式清理进程间通信残留

flush_memory()

LORA_PATH = "./outputs/EWOLiyingZhao_Flux_Final_v7.safetensors"
BASE_MODEL = "./models/FLUX.1-dev"
TRIGGER_WORD = "EWOLiyingZhao"
PROMPT = f"{TRIGGER_WORD} with long hair, wearing a blue shirt, smiling, 8k resolution."
OUTPUT_NAME = "liying_zhao_v2_test.png"

def run_inference():
    if not os.path.exists(LORA_PATH):
        print(f"❌ 错误：找不到文件 {LORA_PATH}")
        return

    print(f"⏳ 5090 正在构建 Flux 管道...")
    
    # 针对 5090 优化：使用 low_cpu_mem_usage 降低加载负荷
    pipe = FluxPipeline.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.bfloat16,
        use_safetensors=True,
        low_cpu_mem_usage=True
    )

    # 针对 32GB 显卡的黄金法则
    pipe.enable_model_cpu_offload()



    print(f"🎨 注入新训练的 LoRA...")

    # 增加 adapter_name 明确注入路径
    pipe.load_lora_weights(LORA_PATH)

    # # 将其替换为手动控制强度的逻辑
    # pipe.load_lora_weights(LORA_PATH, adapter_name="yusaku_lora")
    # # 将强度从默认的 1.0 降低到 0.5 甚至 0.1
    # pipe.set_adapters(["yusaku_lora"], adapter_weights=[0.3])



    print("⚡ 启动首步推理 (由于首步涉及算子初始化，请耐心等待 30-60 秒)...")
    
    with torch.inference_mode():
        image = pipe(
            prompt=PROMPT,
            height=1024,
            width=1024,
            guidance_scale=3.5, 
            num_inference_steps=28,
            max_sequence_length=512
        ).images[0]

    image.save(OUTPUT_NAME)
    print(f"✅ 成功！图片保存在: {os.path.abspath(OUTPUT_NAME)}")

if __name__ == "__main__":
    run_inference()

⏳ 5090 正在构建 Flux 管道...


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.93it/s]


🎨 注入新训练的 LoRA...
⚡ 启动首步推理 (由于首步涉及算子初始化，请耐心等待 30-60 秒)...


100%|██████████| 28/28 [00:47<00:00,  1.69s/it]


✅ 成功！图片保存在: /home/yusaku/Workspace/人脸FineTuneWorkspace/kohya_ss/liying_zhao_v2_test.png
